<a href="https://colab.research.google.com/github/JaiderSantos/metodos1_JaiderSantos_NicolasGomez/blob/main/Parcial2/Parcial_2_Algebra_Lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import sympy as sym
import numpy as np
from scipy import integrate
import math as math
import matplotlib.pyplot as plt
from scipy.special import roots_legendre
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import copy

C = np.array([8.51, 10.68, 12.24, 13.66, 15.37, 17.15, 19.66, 24.69])
Ca = 1000000


def func_de_retorno(C, w, Ca):

    w = np.resize(w, C.shape)
    Pro_elem = w * C
    menos1 = Pro_elem - 1
    RTA = np.min(Ca * (menos1))
    return RTA


def func_de_retorno_max(C, w, Ca):

    w = np.resize(w, C.shape)
    Pro_elem = w * C
    menos1 = Pro_elem - 1
    RTA = Ca * np.max(menos1)
    return RTA


class Robot:
    def __init__(self, f, Id, T):
        self.Id = Id
        self.f = f
        self.h = np.zeros(T)
        for i in range(T):
            self.h[i] = np.random.uniform(1.0, 100.0)
        self.h /= np.sum(self.h)

        self.r = np.random.rand(len(C))
        self.rate = 0.05


    def Mutate(self):
        self.r += np.random.normal(loc=0., scale=self.rate, size=len(self.r))

    def GetR(self):
        return self.r

    def SetFitness(self):
        self.Fitness = self.f(C, self.GetR(), Ca)


        if self.Fitness > 0.05:
            self.rate = 0.005

    def GetFitness(self):
        return self.Fitness


def GetRobots(N, tam):
    Robots = []

    for i in range(N):
        r = Robot(func_de_retorno, i, tam)
        Robots.append(r)

    return Robots


Robots = np.array(GetRobots(500, len(C)))


def GetRobots(N, tam):
    Robots = []

    for i in range(N):
        r = Robot(func_de_retorno, i, tam)
        Robots.append(r)

    return Robots


Robots = np.array(GetRobots(50, len(C)))

for i in range(len(Robots)):
    print(Robots[i].h)




def Genetic(Robots, epochs = 500):



    N = int(0.5*len(Robots))


    Fitness = np.array([])

    for e in range(int(epochs)):


        for i,p in enumerate(Robots):
            p.Mutate()
            p.SetFitness()





        scores = [ (p.Fitness, p) for p in Robots ]
        scores.sort(key = lambda x: x[0], reverse = False)


        Temp = [r[1] for i,r in enumerate(scores) if i < N ]

        for i,r in enumerate(Robots):
            j = i%N
            Robots[i] = copy.deepcopy(Temp[j])



        Fitness_ = [  r.GetFitness() for i,r in enumerate(Robots) ]


        Fitness = np.append(Fitness,np.mean(Fitness_))

        if Fitness[-1] < 0.0001:
            break


        if e % 10 == 0:
            clear_output(wait=True)

            ax,ax1 = Plotter(e)
            for i,p in enumerate(Robots):
                ax.scatter(p.GetR()[0],p.GetR()[1],marker='.',color='r',label='Id {}'.format(p.Id))
            ax.set_xlim(-5,5)
            ax.set_ylim(-5,5)


            ax1.set_title('Fitness function')
            ax1.plot(Fitness,color='k',label='Id {}, {:.2f}, {:.2f}, {:.2f}'.format(Robots[0].Id, Robots[0].GetR()[0],Robots[0].GetR()[1],Robots[0].GetR()[2]))
            ax1.set_xlabel('Epoch')
            ax1.legend(loc=1)

            plt.show()
Genetic(Robots)

params = np.array([ p.GetR() for i, p in enumerate(Robots)]).mean(axis=0)

print(params)



Pesos = np.array([0.185, 0.152, 0.137, 0.125, 0.116, 0.107, 0.096, 0.082])
print(Pesos)
print(Pesos.sum())
print()
# ganacia minima
min_ganancia = func_de_retorno(C, Pesos, Ca)
print("En el peor de los casos el retorno seriade",min_ganancia)
#pocible maximo del retorno de inverción
max_ganancia=func_de_retorno_max(C,Pesos,Ca)
print("el retorno maximo posible seria de",max_ganancia)

#cuanto se invierte en cada opcion
invercion=np.zeros(len(Pesos))
for i in range(len(Pesos)):
  invercion[i]=Pesos[i]*Ca
print(invercion)
Total=np.sum(invercion)
print(Total)




[0.18181639 0.03045363 0.20502222 0.20646555 0.01212808 0.10078726
 0.1436682  0.11965868]
[0.18045868 0.12202281 0.07441394 0.21029781 0.08317322 0.07780743
 0.10244384 0.14938226]
[0.17689379 0.15376506 0.04613931 0.1523633  0.08355676 0.15338614
 0.20662275 0.02727289]
[0.1929948  0.12384657 0.16979228 0.06727042 0.16543974 0.0655635
 0.01850559 0.1965871 ]
[0.17483967 0.21307765 0.15305551 0.0094286  0.08919856 0.10687206
 0.13251498 0.12101298]
[0.18348204 0.08537329 0.16007156 0.04405968 0.10337466 0.19739696
 0.08767854 0.13856325]
[0.02204896 0.03826634 0.07938745 0.10831871 0.08198374 0.32215251
 0.06866242 0.27917987]
[0.28464837 0.00388492 0.07326948 0.00735854 0.21135472 0.19714903
 0.15344041 0.06889453]
[0.06811197 0.15157884 0.03625156 0.07975889 0.15728539 0.15517859
 0.22101121 0.13082353]
[0.09594091 0.1779443  0.15453059 0.16309401 0.1647661  0.16712106
 0.02865708 0.04794595]
[0.04296837 0.0751848  0.08294396 0.15362165 0.20963364 0.21466771
 0.07294668 0.14803319]


No sería rentable realizar esta inversión debido al que el riesgo de pérdida frente a la baja posibilidad de ganancia siendo más probable perder dinero que ganarlo ya que solo se puede dar uno de los casos a la vez por lo que al distribuir el dinero entre los posibles resultados con el fin de obtener una mayor ganancia o una ganancia segura. Se está perdiendo dinero esto se puede ver mejor si analizamos el mejor de los casos en retorno que es de 1024580 lo que da como margen de ganancia neto tan solo 24580 pesos sobre el 1000000 inicialmente invertido.  